In [ ]:
import networkx as nx
import operator
import pylab
import random
import sys

import time
import numpy as np
import osmnx as ox
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
from IPython.display import clear_output
import matplotlib.cm as cm
import matplotlib.colors as colors
from IPython.display import Image
from pprint import pprint
import folium
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
def robustness_analysis(g, node_classifier, recalculate=False):
    m = node_classifier(g)
    l = sorted(m.items(), key=operator.itemgetter(1), reverse=True)
    x = []
    y = []
    
    
    sbg = [g.subgraph(c).copy() for c in nx.connected_components(g)]
    largest_component = max(sbg, key=lambda c: sum(d["length"] for u, v, d in c.edges(data=True)))
    
    
    n = len(g.nodes())
    tl=sum(d["length"] for u, v, d in g.edges(data=True))
    x.append(0)
    y.append(sum(d["length"] for u, v, d in largest_component.edges(data=True)) * 1. / tl)
    r = 0.0
    for i in range(1, n-1):
        g.remove_node(l.pop(0)[0])
        if recalculate:
            m = node_classifier(g)
            l = sorted(m.items(), key=operator.itemgetter(1),reverse=True)
            
        sbg = [g.subgraph(c).copy() for c in nx.connected_components(g)]
        largest_component = max(sbg, key=lambda c: sum(d["length"] for u, v, d in c.edges(data=True)))        
        x.append(i * 1. / n)
        r += sum(d["length"] for u, v, d in largest_component.edges(data=True)) * 1. / tl
        y.append(sum(d["length"] for u, v, d in largest_component.edges(data=True)) * 1. / tl)
    print(r,tl,n)
    return x, y, r / n

In [ ]:
def robustness_analysis_apl(g, node_classifier, recalculate=False):
    m = node_classifier(g)
    l = sorted(m.items(), key=operator.itemgetter(1), reverse=True)
    x = []
    y = []

    average_path_length = 0.0
    number_of_components = 0
    n = len(g.nodes())
    
    
    
    sbg = [g.subgraph(c).copy() for c in nx.connected_components(g)]
    for sg in sbg:
        average_path_length += nx.average_shortest_path_length(sg)
        number_of_components += 1

    average_path_length = average_path_length / number_of_components
    initial_apl = average_path_length

    x.append(0)
    y.append(average_path_length * 1. / initial_apl)
    r = 0.0
    for i in range(1, n-1):
        g.remove_node(l.pop(0)[0])
        if recalculate:
            m = node_classifier(g)
            l = sorted(m.items(), key=operator.itemgetter(1),
                       reverse=True)

        average_path_length = 0.0
        number_of_components = 0

        
        
        sbg = [g.subgraph(c).copy() for c in nx.connected_components(g)]
        for sg in sbg:
            if len(sg.nodes()) > 1:
                average_path_length += nx.average_shortest_path_length(sg)
            number_of_components += 1

        average_path_length = average_path_length / number_of_components

        x.append(i * 1. / n)
        r += average_path_length * 1. / initial_apl
        y.append(average_path_length * 1. / initial_apl)
    return x, y, r / n


In [ ]:

def random_ranking(g):
    nodes = g.nodes()
    values = g.nodes()
    values = list(values)
    random.shuffle(values)    
    
    return dict(zip(nodes, values))


In [ ]:
def get_seeds(G_bike, G_drive, pairs):
    """Get x pairs of random pairs of nodes in the bike and street layer.

    Parameters
    ----------
    G_bike : type
        Description of parameter `G_bike`.
    G_drive : type
        Description of parameter `G_drive`.
    pairs : type
        Description of parameter `pairs`.

    Returns
    -------
    type
        Description of returned object.

    """
    seeds_bike = []
    seeds_car = []
    u = 0
    for u in range(pairs):
        i = random.choice(list(G_bike.nodes(data=True)))
        j = random.choice(list(G_bike.nodes(data=True)))
        u = ox.distance.nearest_nodes(G_drive, i[1]['x'], i[1]['y'])
        v = ox.distance.nearest_nodes(G_drive, j[1]['x'], j[1]['y'])
        if u != v or i[0] != j[0]:
            seeds_car.append((u, v))
            seeds_bike.append((i[0], j[0]))
        else:
            i = random.choice(list(G_bike.nodes(data=True)))
            j = random.choice(list(G_bike.nodes(data=True)))
            u = ox.distance.nearest_nodes(G_drive,  i[1]['x'], i[1]['y'])
            v = ox.distance.nearest_nodes(G_drive,  j[1]['x'], j[1]['y'])
    return seeds_bike, seeds_car



In [ ]:
#euclidean distance
def euclidean_dist_vec(y1, x1, y2, x2):
    '''
    Calculate the euclidean distance between two points.
    '''
    
    #distance = ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5  #use this one if the graph is projected
    distance = ox.distance.great_circle_vec(y1, x1, y2, x2)  #use this one if the graph is not projected
    #print('euclidean distance=',distance)
    return distance

In [ ]:
#shortest path
def get_travel_distance(G, u_v):
    """Find the shortest path between two nodes and calculate the travel distance.

    Parameters
    ----------
    G : nx.Graph
        Networkx graph.
    u_v : touple
        Touple containing the origin and destination for the path.

    Returns
    -------
    int
        Distance for the path between 'u' to 'v'.

    """
    path = nx.shortest_path(G, u_v[0], u_v[1], weight='length')
    distance = 0
    for i, j in zip(path[:-1], path[1:]):
        #distance += float(G[i][j][0]['length'])
        try:
            distance += float(G[i][j][0]['length'])
        except:
            distance += euclidean_dist_vec(G.nodes[i]['y'], G.nodes[i]['x'], G.nodes[j]['y'], G.nodes[j]['x'])
            print('Error: {} {}, distance: {}'.format(G, u_v,euclidean_dist_vec(G.nodes[i]['y'],G.nodes[i]['x'], G.nodes[j]['y'], G.nodes[j]['x'])))
    #print('travel distance=',distance)
    return distance

In [ ]:

def calculate_directness(G_bike, G_drive, name, seeds_bike, car_value):
    d_ij_b = []
    d_ij_s = []
    print('Calculating {}'.format(name))
    try:
        G_bike.add_edge(int(row['i']), int(row['j']), length=euclidean_dist_vec(G_bike.nodes[row['i']]['y'],G_bike.nodes[row['i']]['x'], G_bike.nodes[row['j']]['y'], G_bike.nodes[row['j']]['x']))
    except:
        pass
        
        avg_bike = [] 
        for i_j in seeds_bike:
            if nx.has_path(G_bike, i_j[0], i_j[1]):
                euclidean_distance = euclidean_dist_vec(G_bike.nodes[i_j[0]]['y'], G_bike.nodes[i_j[0]]['x'], G_bike.nodes[i_j[1]]['y'], G_bike.nodes[i_j[0]]['x'])
                bike_distance = get_travel_distance(G_bike, i_j)
                #avg_bike.append(euclidean_distance)
                avg_bike.append(euclidean_distance/bike_distance)
            else:
                avg_bike.append(0)
        bike_value = np.average(avg_bike)
        d_ij_b.append(bike_value)
        d_ij_s.append(car_value)
        #print('bike_distance=',bike_distance)
        print('{} Efficiency: {}/{}'.format(name, round(bike_value, 3), round(car_value, 3)))
        
    #print('Efficiency: {}/{}'.format(round(bike_value, 3), round(car_value, 3)))
    return d_ij_b, G_bike




In [ ]:

ox.config(log_console=True, use_cache=True,overpass_settings='[out:json][timeout:180][date:"2021-04-01T00:00:00Z"]')



cf1 = '["highway"~"cycleway"]["bicycle"!~"no"]'
cf2 = '["cycleway"]["cycleway"!~"no"]'
cf3 = '["cycleway:right"]["cycleway:right"!~"no"]'
cf4 = '["cycleway:left"]["cycleway:left"!~"no"]'
cf5 = '["cycleway:both"]["cycleway:both"!~"no"]'
cf6 = '["bicycle"~"designated"]'
cf7 = '["oneway:bicycle"]'

bbox1 = (45.4976,45.46148,9.2253,9.1642)

G_21_1 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf1)
G_21_2 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf2)
G_21_3 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf3)
G_21_4 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf4)
G_21_5 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf5)
G_21_6 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf6)
G_21_7 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf7)



G_21_11 = nx.compose(G_21_1,  G_21_2)
G_21_12 = nx.compose(G_21_11, G_21_3)
G_21_13 = nx.compose(G_21_12, G_21_4)
G_21_14 = nx.compose(G_21_13, G_21_5)
G_21_15 = nx.compose(G_21_14, G_21_6)
G_21_16 = nx.compose(G_21_15, G_21_7)

G_21_21 = ox.simplification.consolidate_intersections(ox.project_graph(G_21_16,to_crs='EPSG:3812'), tolerance=15, rebuild_graph=True, dead_ends=True, reconnect_edges=True)
G_21_22 = ox.project_graph(G_21_21, to_crs='EPSG:4326')

fig, axb1 = ox.plot_graph(G_21_22, node_size=0,bgcolor='black',edge_color='red',edge_linewidth=0.5)


In [ ]:
ox.config(log_console=True, use_cache=True,overpass_settings='[out:json][timeout:180][date:"2020-01-01T00:00:00Z"]')



cf1 = '["highway"~"cycleway"]["bicycle"!~"no"]'
cf2 = '["cycleway"]["cycleway"!~"no"]'
cf3 = '["cycleway:right"]["cycleway:right"!~"no"]'
cf4 = '["cycleway:left"]["cycleway:left"!~"no"]'
cf5 = '["cycleway:both"]["cycleway:both"!~"no"]'
cf6 = '["bicycle"~"designated"]'
cf7 = '["oneway:bicycle"]'

bbox1 = (45.4976,45.46148,9.2253,9.1642)

G_20_1 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf1)
G_20_2 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf2)
G_20_3 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf3)
G_20_4 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf4)
G_20_6 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf6)
G_20_7 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf7)

G_20_11 = nx.compose(G_20_1,  G_20_2)
G_20_12 = nx.compose(G_20_11, G_20_3)
G_20_13 = nx.compose(G_20_12, G_20_4)
G_20_15 = nx.compose(G_20_13, G_20_6)
G_20_16 = nx.compose(G_20_15, G_20_7)

G_20_21 = ox.simplification.consolidate_intersections(ox.project_graph(G_20_16,to_crs='EPSG:3812'), tolerance=15, rebuild_graph=True, dead_ends=True, reconnect_edges=True)
G_20_22 = ox.project_graph(G_20_21, to_crs='EPSG:4326')

fig, axb2 = ox.plot_graph(G_20_22, node_size=0,bgcolor='black',edge_color='white',edge_linewidth=0.5)


In [ ]:
# get 2020-01-01 car networks(.graphml file) of Milano

ox.config(overpass_settings='[out:json][timeout:180][date:"2021-04-01T00:00:00Z"]')


bbox1 = (45.4976,45.46148,9.2253,9.1642)

G_21_d1 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  network_type='drive',simplify=False, retain_all=False, 
                                  truncate_by_edge=False, clean_periphery=True, custom_filter=None)



#ox.io.save_graphml(G_20_d3, filepath='data_indicator/Milano/Milano_20_drive.graphml', gephi=False, encoding='utf-8')
print('Milano 21_drive_net saved')


In [ ]:
# get 2020-01-01 car networks(.graphml file) of Milano

ox.config(overpass_settings='[out:json][timeout:180][date:"2020-01-01T00:00:00Z"]')


bbox1 = (45.4976,45.46148,9.2253,9.1642)

G_20_d1 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  network_type='drive',simplify=False, retain_all=False, 
                                  truncate_by_edge=False, clean_periphery=True, custom_filter=None)



#ox.io.save_graphml(G_20_d3, filepath='data_indicator/Milano/Milano_20_drive.graphml', gephi=False, encoding='utf-8')
print('Milano 20_drive_net saved')


In [ ]:
#calculate the network efficiency(connectedness) of networks (table 5.1)

g = ox.utils_graph.get_digraph(G_21_21, weight='length')
g_21 = g.to_undirected()

n = len(g_21)
denom = n * (n - 1)
if denom != 0:
    lengths = nx.all_pairs_dijkstra_path_length(g_21, weight="weight")
    g_eff = 0
    for source, targets in lengths:
        for target, distance in targets.items():
            if distance > 0:
                g_eff += (1 / distance)
    g_eff /= denom
    # g_eff = sum(1 / d for s, tgts in lengths
    #                   for t, d in tgts.items() if d > 0) / denom
else:
    g_eff = 0
    
print('eff of 21 Milano {}'.format(g_eff))

g = ox.utils_graph.get_digraph(G_20_21, weight='length')
g_20 = g.to_undirected()

n = len(g_20)
denom = n * (n - 1)
if denom != 0:
    lengths = nx.all_pairs_dijkstra_path_length(g_20, weight="weight")
    g_eff = 0
    for source, targets in lengths:
        for target, distance in targets.items():
            if distance > 0:
                g_eff +=( 1 / distance)
    g_eff /= denom
    # g_eff = sum(1 / d for s, tgts in lengths
    #                   for t, d in tgts.items() if d > 0) / denom
else:
    g_eff = 0
    
print('eff of 20 Milano {}'.format(g_eff))

In [ ]:
for i in range(1,6):

    # calculate the directness and connectedness of 2021 bike networks of Milano
    '''
    Right now there is a bug about loading .graphml file which is saved from intersections consolidated graph, 
    so the graph has to be used at once. But the bug was recently fixed and will appear in the next release, 
    targeted for early May 2021, just right after the paper written. So maybe graphs can be gotten by: 

    G_bike = ox.load_graphml('data_indicator/Milano/Milano_21_bike.graphml')


    '''
    print('calculating indicators of 2021 bike')

    seeds = 1000# how many node pairs are going to be chosen

    G_drive = G_21_d1
    G_drive = ox.get_undirected(G_drive)

    avg_street = []
    avg_street_1=[]
    avg_street_2=[]
    G_bike = G_21_16
    G_bike = ox.get_undirected(G_bike)   
    seeds_bike, seeds_car = get_seeds(G_bike, G_drive, seeds) 
    for u_v in seeds_car:
        euclidean_distance = euclidean_dist_vec(G_drive.nodes[u_v[0]]['y'],G_drive.nodes[u_v[0]]['x'], G_drive.nodes[u_v[1]]['y'], G_drive.nodes[u_v[0]]['x'])
        travel_distance = get_travel_distance(G_drive, u_v)
        if travel_distance==0:
            euclidean_distance=0.0
            travel_distance=1.0
        else:
            result=travel_distance
            avg_street.append(euclidean_distance/travel_distance)


    car_value = np.average(avg_street)  # Average efficiency in the car layer
    d_ij_b, G_bike = calculate_directness(G_bike, G_drive, 'Milano', seeds_bike, car_value)




In [ ]:
for i in range(1,6):
    # calculate the directness and connectedness of 2020 bike networks of Milano
    '''
    Right now there is a bug about loading .graphml file which is saved from intersections consolidated graph, 
    so the graph has to be used at once. But the bug was recently fixed and will appear in the next release, 
    targeted for early May 2021, just right after the paper written. So maybe graphs can be gotten by: 

    G_bike = ox.load_graphml('data_indicator/Milano/Milano_20_bike.graphml')


    '''
    print('calculating indicators of 2020 bike')

    seeds = 1000# how many node pairs are going to be chosen

    G_drive = G_20_d1
    G_drive = ox.get_undirected(G_drive)

    avg_street = []
    avg_street_1=[]
    avg_street_2=[]
    G_bike = G_20_16
    G_bike = ox.get_undirected(G_bike)   
    seeds_bike, seeds_car = get_seeds(G_bike, G_drive, seeds) 
    for u_v in seeds_car:
        euclidean_distance = euclidean_dist_vec(G_drive.nodes[u_v[0]]['y'],G_drive.nodes[u_v[0]]['x'], G_drive.nodes[u_v[1]]['y'], G_drive.nodes[u_v[0]]['x'])
        travel_distance = get_travel_distance(G_drive, u_v)
        if travel_distance==0:
            euclidean_distance=0.0
            travel_distance=1.0
        else:
            result=travel_distance
            avg_street.append(euclidean_distance/travel_distance)


    car_value = np.average(avg_street)  # Average efficiency in the car layer
    d_ij_b, G_bike = calculate_directness(G_bike, G_drive, 'Milano', seeds_bike, car_value)




In [ ]:

d_original = ox.stats.basic_stats(G_21_16)
print('Milano 20_d_original:\n{}'.format(d_original))


In [ ]:

d_original = ox.stats.basic_stats(G_20_16)
print('Milano 20_d_original:\n{}'.format(d_original))


In [ ]:

ox.config(log_console=True, use_cache=True,overpass_settings='[out:json][timeout:180][date:"2021-04-01T00:00:00Z"]')



cf1 = '["highway"~"cycleway"]["bicycle"!~"no"]'
cf2 = '["cycleway"]["cycleway"!~"no"]'
cf3 = '["cycleway:right"]["cycleway:right"!~"no"]'
cf4 = '["cycleway:left"]["cycleway:left"!~"no"]'
cf5 = '["cycleway:both"]["cycleway:both"!~"no"]'
cf6 = '["bicycle"~"designated"]'
cf7 = '["oneway:bicycle"]'

bbox1 = (45.4976,45.46148,9.2253,9.1642)

G_21_1 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf1)
G_21_2 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf2)
G_21_3 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf3)
G_21_4 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf4)
G_21_5 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf5)
G_21_6 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf6)
G_21_7 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf7)



G_21_11 = nx.compose(G_21_1,  G_21_2)
G_21_12 = nx.compose(G_21_11, G_21_3)
G_21_13 = nx.compose(G_21_12, G_21_4)
G_21_14 = nx.compose(G_21_13, G_21_5)
G_21_15 = nx.compose(G_21_14, G_21_6)
G_21_16 = nx.compose(G_21_15, G_21_7)
G_21_16 = ox.simplification.simplify_graph(G_21_16)

G_21_21 = ox.simplification.consolidate_intersections(ox.project_graph(G_21_16,to_crs='EPSG:3812'), tolerance=15, rebuild_graph=True, dead_ends=True, reconnect_edges=True)
G_21_22 = ox.project_graph(G_21_21, to_crs='EPSG:4326')

fig, axb1 = ox.plot_graph(G_21_22, node_size=0,bgcolor='black',edge_color='red',edge_linewidth=0.5)


In [ ]:
ox.config(log_console=True, use_cache=True,overpass_settings='[out:json][timeout:180][date:"2020-01-01T00:00:00Z"]')



cf1 = '["highway"~"cycleway"]["bicycle"!~"no"]'
cf2 = '["cycleway"]["cycleway"!~"no"]'
cf3 = '["cycleway:right"]["cycleway:right"!~"no"]'
cf4 = '["cycleway:left"]["cycleway:left"!~"no"]'
cf5 = '["cycleway:both"]["cycleway:both"!~"no"]'
cf6 = '["bicycle"~"designated"]'
cf7 = '["oneway:bicycle"]'

bbox1 = (45.4976,45.46148,9.2253,9.1642)

G_20_1 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf1)
G_20_2 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf2)
G_20_3 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf3)
G_20_4 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf4)
G_20_6 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf6)
G_20_7 = ox.graph.graph_from_bbox(45.4976,45.46148,9.2253,9.1642,  simplify=False, retain_all=True, truncate_by_edge=False, clean_periphery=True, custom_filter=cf7)

G_20_11 = nx.compose(G_20_1,  G_20_2)
G_20_12 = nx.compose(G_20_11, G_20_3)
G_20_13 = nx.compose(G_20_12, G_20_4)
G_20_15 = nx.compose(G_20_13, G_20_6)
G_20_16 = nx.compose(G_20_15, G_20_7)
G_20_16 = ox.simplification.simplify_graph(G_20_16)

G_20_21 = ox.simplification.consolidate_intersections(ox.project_graph(G_20_16,to_crs='EPSG:3812'), tolerance=15, rebuild_graph=True, dead_ends=True, reconnect_edges=True)
G_20_22 = ox.project_graph(G_20_21, to_crs='EPSG:4326')

fig, axb2 = ox.plot_graph(G_20_22, node_size=0,bgcolor='black',edge_color='white',edge_linewidth=0.5)


In [ ]:
#g = ox.utils_graph.get_digraph(G_21_16, weight='length')
g_21 = G_21_21.to_undirected()

n = len(g_21)
denom = n * (n - 1)
if denom != 0:
    lengths = nx.all_pairs_dijkstra_path_length(g_21, weight="weight")
    g_eff = 0
    for source, targets in lengths:
        for target, distance in targets.items():
            if distance > 0:
                g_eff += 1 / distance
    g_eff /= denom
    # g_eff = sum(1 / d for s, tgts in lengths
    #                   for t, d in tgts.items() if d > 0) / denom
else:
    g_eff = 0
    
print('eff of 21 Milano {}'.format(g_eff))

#g = ox.utils_graph.get_digraph(G_20_16, weight='length')
g_20 = G_20_21.to_undirected()

n = len(g_20)
denom = n * (n - 1)
if denom != 0:
    lengths = nx.all_pairs_dijkstra_path_length(g_20, weight="weight")
    g_eff = 0
    for source, targets in lengths:
        for target, distance in targets.items():
            if distance > 0:
                g_eff += 1 / distance
    g_eff /= denom
    # g_eff = sum(1 / d for s, tgts in lengths
    #                   for t, d in tgts.items() if d > 0) / denom
else:
    g_eff = 0
    
print('eff of 20 Milano {}'.format(g_eff))

In [ ]:
apl=True

if apl==True:
    analysis_method = robustness_analysis_apl
    method='apl'
else:
    analysis_method = robustness_analysis
    method='com'
        
outfile='data_indicator/Milano/Robustness/part2/part2_b21_{}.png'.format(method)

recalculate=False

g = ox.utils_graph.get_digraph(G_21_16, weight='length')
g = g.to_undirected()
#g = ox.utils_graph.get_undirected(G_bspf)




x1, y1, vd = analysis_method(g.copy(), nx.degree_centrality, recalculate)
x2, y2, vb = analysis_method(g.copy(), nx.betweenness_centrality, recalculate)
x3, y3, vc = analysis_method(g.copy(), nx.closeness_centrality, recalculate)
x5, y5, vr = analysis_method(g.copy(), random_ranking)

if not apl:
    pylab.figure(1, dpi=1080)
    pylab.xlabel(r"Fraction of vertices removed ($\rho$)")
    pylab.ylabel(r"Fractional size of largest component ($\sigma$)")
else:
    pylab.figure(1, dpi=1080)
    pylab.xlabel(r"Fraction of vertices removed ($\rho$)")
    pylab.ylabel(r"Average path length ($\sigma$)")

pylab.plot(x1, y1, "b-", alpha=0.6, linewidth=2.0)
pylab.plot(x2, y2, "g-", alpha=0.6, linewidth=2.0)
pylab.plot(x3, y3, "r-", alpha=0.6, linewidth=2.0)
pylab.plot(x5, y5, "k-", alpha=0.6, linewidth=2.0)

# Generate csv file
import numpy as np

pylab.legend((r"Degree ($R = %4.3f$)" % vd,
              "Betweenness ($R = %4.3f$)" % vb,
              "Closeness ($R = %4.3f$)" % vc,
              "Random ($R = %4.3f$)" % vr),
            loc="upper right", shadow=False)

pylab.savefig(outfile, format="png")
pylab.close(1)

matrix = np.matrix([x1, y1, y2, y3, y5])
filename = outfile.rsplit(".", 1)[0] + ".csv"
header = " , degree, betweeness, closeness, random"
separator = ", "

np.savetxt(filename, matrix.transpose(), fmt="%2.5f", delimiter=separator,header=header, comments="")



In [ ]:
apl=True

if apl==True:
    analysis_method = robustness_analysis_apl
    method='apl'
else:
    analysis_method = robustness_analysis
    method='com'
        
outfile='data_indicator/Milano/Robustness/part2/part2_b20_{}.png'.format(method)

recalculate=False

g = ox.utils_graph.get_digraph(G_20_16, weight='length')
g = g.to_undirected()
#g = ox.utils_graph.get_undirected(G_bspf)




x1, y1, vd = analysis_method(g.copy(), nx.degree_centrality, recalculate)
x2, y2, vb = analysis_method(g.copy(), nx.betweenness_centrality, recalculate)
x3, y3, vc = analysis_method(g.copy(), nx.closeness_centrality, recalculate)
x5, y5, vr = analysis_method(g.copy(), random_ranking)

if not apl:
    pylab.figure(1, dpi=1080)
    pylab.xlabel(r"Fraction of vertices removed ($\rho$)")
    pylab.ylabel(r"Fractional size of largest component ($\sigma$)")
else:
    pylab.figure(1, dpi=1080)
    pylab.xlabel(r"Fraction of vertices removed ($\rho$)")
    pylab.ylabel(r"Average path length ($\sigma$)")

pylab.plot(x1, y1, "b-", alpha=0.6, linewidth=2.0)
pylab.plot(x2, y2, "g-", alpha=0.6, linewidth=2.0)
pylab.plot(x3, y3, "r-", alpha=0.6, linewidth=2.0)
pylab.plot(x5, y5, "k-", alpha=0.6, linewidth=2.0)

# Generate csv file
import numpy as np

pylab.legend((r"Degree ($R = %4.3f$)" % vd,
              "Betweenness ($R = %4.3f$)" % vb,
              "Closeness ($R = %4.3f$)" % vc,
              "Random ($R = %4.3f$)" % vr),
            loc="upper right", shadow=False)

pylab.savefig(outfile, format="png")
pylab.close(1)

matrix = np.matrix([x1, y1, y2, y3, y5])
filename = outfile.rsplit(".", 1)[0] + ".csv"
header = " , degree, betweeness, closeness, random"
separator = ", "

np.savetxt(filename, matrix.transpose(), fmt="%2.5f", delimiter=separator,header=header, comments="")



In [ ]:
apl=False

if apl==True:
    analysis_method = robustness_analysis_apl
    method='apl'
else:
    analysis_method = robustness_analysis
    method='com'
        
outfile='data_indicator/Milano/Robustness/part2/part2_b21_{}.png'.format(method)

recalculate=False

g = ox.utils_graph.get_digraph(G_21_16, weight='length')
g = g.to_undirected()
#g = ox.utils_graph.get_undirected(G_bspf)




x1, y1, vd = analysis_method(g.copy(), nx.degree_centrality, recalculate)
x2, y2, vb = analysis_method(g.copy(), nx.betweenness_centrality, recalculate)
x3, y3, vc = analysis_method(g.copy(), nx.closeness_centrality, recalculate)
x5, y5, vr = analysis_method(g.copy(), random_ranking)

if not apl:
    pylab.figure(1, dpi=1080)
    pylab.xlabel(r"Fraction of vertices removed ($\rho$)")
    pylab.ylabel(r"Fractional size of largest component ($\sigma$)")
else:
    pylab.figure(1, dpi=1080)
    pylab.xlabel(r"Fraction of vertices removed ($\rho$)")
    pylab.ylabel(r"Average path length ($\sigma$)")

pylab.plot(x1, y1, "b-", alpha=0.6, linewidth=2.0)
pylab.plot(x2, y2, "g-", alpha=0.6, linewidth=2.0)
pylab.plot(x3, y3, "r-", alpha=0.6, linewidth=2.0)
pylab.plot(x5, y5, "k-", alpha=0.6, linewidth=2.0)

# Generate csv file
import numpy as np

pylab.legend((r"Degree ($R = %4.3f$)" % vd,
              "Betweenness ($R = %4.3f$)" % vb,
              "Closeness ($R = %4.3f$)" % vc,
              "Random ($R = %4.3f$)" % vr),
            loc="upper right", shadow=False)

pylab.savefig(outfile, format="png")
pylab.close(1)

matrix = np.matrix([x1, y1, y2, y3, y5])
filename = outfile.rsplit(".", 1)[0] + ".csv"
header = " , degree, betweeness, closeness, random"
separator = ", "

np.savetxt(filename, matrix.transpose(), fmt="%2.5f", delimiter=separator,header=header, comments="")



In [ ]:
apl=False

if apl==True:
    analysis_method = robustness_analysis_apl
    method='apl'
else:
    analysis_method = robustness_analysis
    method='com'
        
outfile='data_indicator/Milano/Robustness/part2/part2_b20_{}.png'.format(method)

recalculate=False

g = ox.utils_graph.get_digraph(G_20_16, weight='length')
g = g.to_undirected()
#g = ox.utils_graph.get_undirected(G_bspf)




x1, y1, vd = analysis_method(g.copy(), nx.degree_centrality, recalculate)
x2, y2, vb = analysis_method(g.copy(), nx.betweenness_centrality, recalculate)
x3, y3, vc = analysis_method(g.copy(), nx.closeness_centrality, recalculate)
x5, y5, vr = analysis_method(g.copy(), random_ranking)

if not apl:
    pylab.figure(1, dpi=1080)
    pylab.xlabel(r"Fraction of vertices removed ($\rho$)")
    pylab.ylabel(r"Fractional size of largest component ($\sigma$)")
else:
    pylab.figure(1, dpi=1080)
    pylab.xlabel(r"Fraction of vertices removed ($\rho$)")
    pylab.ylabel(r"Average path length ($\sigma$)")

pylab.plot(x1, y1, "b-", alpha=0.6, linewidth=2.0)
pylab.plot(x2, y2, "g-", alpha=0.6, linewidth=2.0)
pylab.plot(x3, y3, "r-", alpha=0.6, linewidth=2.0)
pylab.plot(x5, y5, "k-", alpha=0.6, linewidth=2.0)

# Generate csv file
import numpy as np

pylab.legend((r"Degree ($R = %4.3f$)" % vd,
              "Betweenness ($R = %4.3f$)" % vb,
              "Closeness ($R = %4.3f$)" % vc,
              "Random ($R = %4.3f$)" % vr),
            loc="upper right", shadow=False)

pylab.savefig(outfile, format="png")
pylab.close(1)

matrix = np.matrix([x1, y1, y2, y3, y5])
filename = outfile.rsplit(".", 1)[0] + ".csv"
header = " , degree, betweeness, closeness, random"
separator = ", "

np.savetxt(filename, matrix.transpose(), fmt="%2.5f", delimiter=separator,header=header, comments="")

